# HTML 化された決算短信からセグメント情報を抽出する方法を学ぶ

この Notebook で、 Python を用いて HTML 化された決算短信からセグメント情報を抽出する方法を学ぶことができます。

## HTML 化された決算短信とは

HTML 化された決算短信とは、上場会社及び上場 REIT の決算短信における目次以降のページ(目次＋定性的情報＋財務諸表等(財務諸表に関する注記事項を含む))を HTML 化した報告様式です。

![financial_html.png](images/financial_html.png)

セグメント情報は、セグメントを持つ企業が開示しているセグメントごとの収益などの報告です。投資家や情報ベンダーの間でもニーズが高い情報です。

![segment_information](images/segment_information.png)

## Exercies1: 決算短信 HTML ファイルからセグメント情報を抽出する

`data/raw/HTML/セブン＆アイ・ホールディングス`にセブン＆アイ・ホールディングスの HTML 化された決算短信を格納しています。 HTML 化された決算短信から次の手順で情報を抽出します。

1. 「報告セグメント」を含む HTML テーブルを検索する
2. 「報告セグメント」の HTML テーブルから、報告時期を取得する
3. 「報告セグメント」の HTML テーブルから、セグメントの列を取得する
4. 「報告セグメント」の HTML テーブルから、売上・利益の行を取得する
5.  各セグメントの列、売上・利益の行を指定し値を取得する

Exercise2 で実際に抽出を行う、また抽出時エラーが起こったときの原因が特定できるようにするため、 Exercise1 で 1 ~ 5 の処理について詳しく解説を行います。

はじめに、HTML ファイルを読み込みます。

In [1]:
html_path = "../data/raw/HTML/セブン＆アイ・ホールディングス/qualitative.htm"

In [2]:
import dataclasses
import re
import unicodedata
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import Optional, Union, List, Tuple

import pandas as pd
from bs4 import BeautifulSoup
from bs4.element import Tag


def read_financial_result_html(path: Union[str, Path]) -> Optional[BeautifulSoup]:
    """
    決算短信のHTMLをBeautiful Soupに読み込む。

    Parameters
    ----------
    path : str | Path
        決算短信のHTMLファイルのパス

    Returns
    -------
    Optional[BeautifulSoup]
        Beautiful Soupのオブジェクト
    """

    _path = path if isinstance(path, Path) else Path(path)
    text = ""
    if not _path.exists():
        return None

    with _path.open(encoding="utf-8") as r:
        lines = r.readlines()
        # 記載されたテキストの文字種を統一するため、Unicode正規化を行う
        lines = [unicodedata.normalize("NFKC", line) for line in lines]
        text = "".join(lines)

    html = BeautifulSoup(text, features="html.parser")
    return html

In [3]:
html = read_financial_result_html(html_path)

### 1. 「報告セグメント」を含む HTML テーブルを検索する

「報告セグメント」という記載がある HTML テーブルを検索します。決算短信のHTMLには、 `id` など決定的につかえる属性がないので、「報告セグメント」というテキストを頼りにそれが含まれているテーブルを検索します。

![step_001.png](images/step_001.png)

In [4]:
def find_segment_tables(html: BeautifulSoup) -> List[Tag]:
    """
    「報告セグメント」のテキストを含むテーブルを検索する

    Parameters
    ----------
    html: BeautifulSoup
        決算短信のHTMLを読み込んだBeautiful Soupのオブジェクト

    Returns
    -------
    list[Tag]
        「報告セグメント」のテキストを含むテーブルのリスト
    """
    tables = html.find_all("table")  # 1: テーブルを検索
    segment_tables = []
    for t in tables:
        text = t.find(text="報告セグメント")
        if text is not None:  # 2: 「報告セグメント」の記載があることを確認
            td = text.find_previous("td")
            if td is not None and "colspan" in td.attrs: # 3.「報告セグメント」が結合セルで記載されていることを確認
                segment_tables.append(t)

    return segment_tables

In [5]:
segment_tables = find_segment_tables(html)
len(segment_tables)

2

通常は前期と当期のセグメント報告があるため、 2 つテーブルが取得できるはずです。

In [6]:
from IPython.display import HTML


HTML(str(segment_tables[0]))

### 2. 「報告セグメント」の HTML テーブルから、報告時期を取得する

現状では複数のテーブルのうちどちらが当期でどちらが前期なのか判断がつきません。そのため、テーブルの上部にあるテキストまでたどり着きそこから年月を取得します。

![step_002.png](images/step_002.png)

In [7]:
@dataclass
class Period:
    kind: str = ""
    description: str = ""
    begin: Optional[datetime] = None
    end: Optional[datetime] = None


def read_table_period(table: Tag) -> Period:
    """
    セグメント報告のテーブル上部にあるテキストから、報告年月日を取得する

    Parameters
    ----------
    table: Tag
        セグメント報告のテーブル要素

    Returns
    -------
    Period
        報告年月日
    """

    FIND_LIMIT = 3
    PATTERN = re.compile(r".*(前|当).+(\d+年\d+月\d+日).+(\d+年\d+月\d+日)")
    DATE_PATTERN = re.compile(r"\d+年\d+月\d+日")
    period = Period()

    count = 0
    tag = table
    while count < FIND_LIMIT:
        p = tag.find_previous("p")  # 1: テーブルの前要素にある記述をFIND_LIMITまで検索する
        if p is not None:
            text = p.get_text().strip()
            if re.search(PATTERN, text):  # 2: 日付の記述パターンに合致するか判定し、合致すれば年月日を取得する
                period.description = text
                if "前" in text:
                    period.kind = "previous"
                elif "当" in text:
                    period.kind = "current"

                dates = DATE_PATTERN.findall(period.description)
                if len(dates) == 2:
                    from_date = dates[0].strip()
                    period.begin = datetime.strptime(from_date, "%Y年%m月%d日")
                    to_date = dates[1].strip()
                    period.end = datetime.strptime(to_date, "%Y年%m月%d日")
                break
        count += 1
        tag = p

    return period


In [8]:
periods = [read_table_period(table) for table in segment_tables]

当期について、テキストの通り3/1から5/31が取れていることがわかります。

In [9]:
periods[1]

Period(kind='current', description='II 当第1四半期連結累計期間(自 2022年3月1日 至 2022年5月31日)', begin=datetime.datetime(2022, 3, 1, 0, 0), end=datetime.datetime(2022, 5, 31, 0, 0))

### 3.「報告セグメント」のHTMLテーブルから、セグメントの列を取得する

テーブルからは行と列それぞれを指定してデータを取り出します。セグメント情報のテーブルでは、行は勘定、列はセグメントです。はじめにセグメントの列を取り出し、次に売上・利益の行を取得し、最後に行と列を指定してデータを抽出します。

![step_003.png](images/step_003.png)

In [10]:
@dataclass
class Segment:
    order: int = 0
    position: int = 0
    name: str = ""



def read_table_segments(table: Tag) -> list[Segment]:
    """
    セグメント報告のテーブルから、セグメントの位置を取得する

    Parameters
    ----------
    table: Tag
        セグメント報告のテーブル要素

    Returns
    -------
    list[Segment]
        セグメントのリスト
    """

    merged_cells = table.find_all("td", colspan=True)
    SEGMENT_TEXT = "報告セグメント"
    EXCLUDES = re.compile(r".{0,2}計$")
    NORMALIZER = re.compile(r"\s|\r|\n")
    segments = []  # type: list[Segment]

    # 1: 報告セグメントのセルを取得
    segment_title_cells = [cell for cell in merged_cells if SEGMENT_TEXT in cell.text]
    if len(segment_title_cells) == 0:
        return segments
    else:
        segment_title_cell = segment_title_cells[0]
        # 2: 結合セルのサイズから、セグメントの数を特定
        num_segments = int(segment_title_cell.attrs["colspan"])

        # 3: 「報告セグメント」の開始位置を特定
        segment_begin = 0
        skipped = 0
        for cell in segment_title_cell.find_previous("tr").find_all("td"):
            if cell.text.strip() == SEGMENT_TEXT:
                break
            else:
                if "rowspan" not in cell.attrs:
                    segment_begin += 1
                else:
                    skipped += 1

        # 4: 「報告セグメント」の次の行を取得
        segment_row = segment_title_cell.find_next("tr")
        order = 0
        for i, segment_cell in enumerate(segment_row.find_all("td")):
            if i < segment_begin:
                continue
            # 5: 「報告セグメント」の開始位置からセグメント数だけテキストを取得。
            elif i < (segment_begin + num_segments):
                segment_text = NORMALIZER.sub("", segment_cell.text.strip())
                if segment_text and not EXCLUDES.match(segment_text):
                    segments.append(Segment(order, i + skipped, segment_text))
                    order += 1

    return segments

In [11]:
table_segments = [read_table_segments(table) for table in segment_tables]

きちんとセグメント情報が取れていることがわかります。 `order` はセグメントの並び順で、 `position` は実際のテーブル上でのセルの位置です。

In [12]:
table_segments[0]

[Segment(order=0, position=1, name='国内コンビニエンスストア事業'),
 Segment(order=1, position=2, name='海外コンビニエンスストア事業'),
 Segment(order=2, position=3, name='スーパーストア事業'),
 Segment(order=3, position=4, name='百貨店・専門店事業'),
 Segment(order=4, position=5, name='金融関連事業'),
 Segment(order=5, position=6, name='その他の事業')]

### 4. 「報告セグメント」のHTMLテーブルから、売上・利益の行を取得する

続いて、セグメント報告のうち売上・利益の行を取得します。

![step_004.png](images/step_004.png)

In [13]:
@dataclass
class Account:
    order: int = 0
    position: int = 0
    kind: str = ""
    name: str = ""
    unit: int = 1


def read_table_sales_profit(table: Tag) -> list[Account]:
    """
    セグメント報告のテーブルから、売上・利益の勘定の位置を取得する

    Parameters
    ----------
    table: Tag
        セグメント報告のテーブル要素

    Returns
    -------
    list[Account]
        勘定のリスト
    """

    SUM = re.compile(r".*計$")
    NORMALIZER = re.compile(r"\s|\r|\n")
    UNIT_TEXT = "単位"
    unit = 1000000

    merged_cell = table.find_all("td", colspan=True)
    unit_cells = [cell for cell in merged_cell if UNIT_TEXT in cell.text]

    # 1: 単位のセルから単位を取得(デフォルトは百万)
    if len(unit_cells) > 0:
        cell = unit_cells[0]
        if "千円" in cell.text:
            unit = 1000
        elif "十億" in cell.text:
            unit = 1000000000

    # 2: 各行の先頭 = 勘定のリストを取得
    rows = table.find_all("tr")

    skip_rows = 0
    sales = None
    profit = None
    for i, row in enumerate(rows):
        if skip_rows > 0:
            skip_rows -= 1
            continue

        account_cell = row.find_next("td")
        if "rowspan" in account_cell.attrs:
            skip_rows = int(account_cell.attrs["rowspan"]) - 1
        
        account_text = NORMALIZER.sub("", account_cell.text)
        # 3: 勘定名が登場する最初の行を売上とみなす。※「計」が登場した場合は「計」を売上とする
        if account_text:
            if sales is None:
                sales = Account(0, i, "Sales", account_text, unit)
                profit = None
            elif sales and SUM.match(account_text):
                sales = Account(0, i, "Sales", sales.name, unit)
                profit = None
            elif sales and profit is None:
                # 4: 売上の次の行が利益とする
                profit = Account(1, i, "Profit", account_text, unit)

    accounts = []
    if sales is not None:
        accounts.append(sales)
    if profit is not None:
        accounts.append(profit)

    return accounts

In [14]:
table_accounts = [read_table_sales_profit(table) for table in segment_tables]

売上、利益の行が取れていることを確認できます。

In [15]:
table_accounts[0]

[Account(order=0, position=6, kind='Sales', name='営業収益', unit=1000000),
 Account(order=1, position=7, kind='Profit', name='セグメント利益又は損失(△)', unit=1000000)]

### 5. 各セグメントの列、売上・利益の行を指定し値を取得する

列としてのセグメント、行としての勘定を指定してテーブル内の値を取得します。

![step_005.png](images/step_005.png)

In [16]:
def read_segment_sales_profit(
    table: Tag, segment: Segment, account: Account
) -> pd.Series:
    """
    セグメント報告のテーブルから、セグメント、勘定を指定してデータを取得する

    Parameters
    ----------
    table: Tag
        セグメント報告のテーブル要素
    segment: Segment
        セグメントの位置
    account: Account
        勘定の位置

    Returns
    -------
    pd.Series
        指定されたセグメント、勘定のデータ
    """

    # 1: 勘定の位置の行、セグメントの位置のセルを取得する
    cell = table.find_all("tr")[account.position].find_all("td")[segment.position]
    data = {}
    data.update(
        {f"segment_{key}": value for key, value in dataclasses.asdict(segment).items()}
    )
    data.update(
        {f"account_{key}": value for key, value in dataclasses.asdict(account).items()}
    )
    # 桁区切りの除去、△のマイナスへの変換を行う
    value = cell.text.strip().replace("-", "").replace(",", "").replace("△", "-")
    try:
        data["value"] = float(value)
    except ValueError:
        data["value"] = None

    return pd.Series(data)


これまで取得したセグメントと勘定で、データを取り出してみましょう。前期は `table_index = 0` 、当期は `table_index = 1` です。 HTML と比較して、どちらもとれていることを確認してみてください。

In [17]:
table_index = 0
segment_data = []

for s in table_segments[table_index]:
    for a in table_accounts[table_index]:
        segment_data.append(read_segment_sales_profit(segment_tables[table_index], s, a))

segment_df = pd.DataFrame(segment_data)
segment_df

,segment_order,segment_position,segment_name,account_order,account_position,account_kind,account_name,account_unit,value
0,0,1,国内コンビニエンスストア事業,0,6,Sales,営業収益,1000000,217536.0
1,0,1,国内コンビニエンスストア事業,1,7,Profit,セグメント利益又は損失(△),1000000,60573.0
2,1,2,海外コンビニエンスストア事業,0,6,Sales,営業収益,1000000,679296.0
3,1,2,海外コンビニエンスストア事業,1,7,Profit,セグメント利益又は損失(△),1000000,12136.0
4,2,3,スーパーストア事業,0,6,Sales,営業収益,1000000,451684.0
5,2,3,スーパーストア事業,1,7,Profit,セグメント利益又は損失(△),1000000,5843.0
6,3,4,百貨店・専門店事業,0,6,Sales,営業収益,1000000,166636.0
7,3,4,百貨店・専門店事業,1,7,Profit,セグメント利益又は損失(△),1000000,-3442.0
8,4,5,金融関連事業,0,6,Sales,営業収益,1000000,49101.0
9,4,5,金融関連事業,1,7,Profit,セグメント利益又は損失(△),1000000,10431.0


In [18]:
HTML(str(segment_tables[table_index]))

## Exercies2: セグメント情報の抽出が失敗する理由を分析する

セブン＆アイ・ホールディングスではうまくいきましたが他の企業ではどうでしょうか? 以下に、Exercise1で実施した5つのステップをまとめて実行する関数を用意しました。関心のある企業をこの関数で実行してみて、データが取れるかどうか試してみましょう。

In [19]:
class SegmentNotFoundException(Exception):
    """セグメント情報が見つからないエラー"""
    pass


class SegmentTableError():

    def __init__(self, index: int, table: Tag, error: str) -> None:
        """
        エラーが発生した報告セグメントとエラー内容を格納する

        Parameters
        ----------
        index: int
            テーブルの登場順序
        table: Tag
            セグメント報告のテーブル要素
        error: str
            発生したエラー
        """
        self.index = index
        self.table = table
        self.error = error


class ReadSegmentTableException(Exception):
    """報告セグメントのテーブルの読み取りエラー"""

    def __init__(self, message: str, errors: List[SegmentTableError]) -> None:
        """
        エラーが発生した報告セグメントとエラー内容を格納する

        Parameters
        ----------
        message: str
            エラーメッセージ
        errors: List[SegmentTableError]
            エラーの発生したテーブルとエラー内容
        """
        super().__init__(message)
        self.message = message
        self.errors = errors
    
    def __repr__(self) -> str:
        messages = [self.message]
        for e in self.errors:
            messages.append(f"{e.index}番目のテーブル: {e.error}")
        
        messages.append("※N番目のテーブルのHTML要素は `exception.errors[N].table` でアクセスできます。")
        return "\n".join(messages)


def financial_result_to_dataframe(path: Union[str, Path]) -> Tuple[Optional[pd.DataFrame], List[str]]:
    """
    決算短信のHTMLファイルから、セグメント情報をデータフレーム形式で抽出する

    Parameters
    ----------
    path : str | Path
        決算短信のHTMLファイルのパス

    Returns
    -------
    tuple[Optional[pd.DataFrame], list[str]]
        セグメント情報を読み込んだデータフレームと、ログ情報
    """

    html = read_financial_result_html(path)
    if html is None:
        raise FileNotFoundError("HTMLファイルが見つかりませんでした")

    segment_tables = find_segment_tables(html)
    if len(segment_tables) == 0:
        raise SegmentNotFoundException("「報告セグメント」を含むテーブルを発見できませんでした")

    data = []
    errors = []
    completed_count = 0
    for i, table in enumerate(segment_tables):
        period = read_table_period(table)
        if not period.kind:
            errors.append(SegmentTableError(i, table, f"{i}番目のテーブルでは年月日が取得できませんでした"))
            continue

        segments = read_table_segments(table)
        if len(segments) == 0:
            errors.append(SegmentTableError(i, table, f"{i}番目のテーブルではセグメントが取得できませんでした"))
            continue

        accounts = read_table_sales_profit(table)
        if len(accounts) != 2:
            errors.append(SegmentTableError(i, table, f"{i}番目のテーブルでは売上・利益の勘定が取得できませんでした"))
            continue

        period_dict = {
            f"period_{key}": value for key, value in dataclasses.asdict(period).items()
        }
        _period = pd.Series(period_dict)
        invalid = False
        for s in segments:
            for a in accounts:
                _data = read_segment_sales_profit(table, s, a)
                if _data.value is not None:
                    data.append(pd.concat([_period, _data]))
                else:
                    errors.append(SegmentTableError(i, table, f"{i}番目のテーブルではセグメントと勘定から値が取得できませんでした"))
                    invalid = True
                
                if invalid:
                    break
            
            if invalid:
                break

        if invalid:
            continue

        completed_count += 1


    if completed_count == 2:  # 前期と当期の2つからデータが取得できている
        return (pd.DataFrame(data), errors)
    else:
        if completed_count < 2:
            raise ReadSegmentTableException("前期、当期2つのテーブルが取得できませんでした。", errors)
        else:
            raise ReadSegmentTableException("前期、当期2つ以外のセグメント情報が存在します。", errors)


先程のセブン＆アイ・ホールディングスから抽出すると以下の結果になります。

In [20]:
segment_df, errors = financial_result_to_dataframe(html_path)
segment_df

,period_kind,period_description,period_begin,period_end,segment_order,segment_position,segment_name,account_order,account_position,account_kind,account_name,account_unit,value
0,previous,I 前第1四半期連結累計期間(自 2021年3月1日 至 2021年5月31日),2021-03-01,2021-05-31,0,1,国内コンビニエンスストア事業,0,6,Sales,営業収益,1000000,217536.0
1,previous,I 前第1四半期連結累計期間(自 2021年3月1日 至 2021年5月31日),2021-03-01,2021-05-31,0,1,国内コンビニエンスストア事業,1,7,Profit,セグメント利益又は損失(△),1000000,60573.0
2,previous,I 前第1四半期連結累計期間(自 2021年3月1日 至 2021年5月31日),2021-03-01,2021-05-31,1,2,海外コンビニエンスストア事業,0,6,Sales,営業収益,1000000,679296.0
3,previous,I 前第1四半期連結累計期間(自 2021年3月1日 至 2021年5月31日),2021-03-01,2021-05-31,1,2,海外コンビニエンスストア事業,1,7,Profit,セグメント利益又は損失(△),1000000,12136.0
4,previous,I 前第1四半期連結累計期間(自 2021年3月1日 至 2021年5月31日),2021-03-01,2021-05-31,2,3,スーパーストア事業,0,6,Sales,営業収益,1000000,451684.0
5,previous,I 前第1四半期連結累計期間(自 2021年3月1日 至 2021年5月31日),2021-03-01,2021-05-31,2,3,スーパーストア事業,1,7,Profit,セグメント利益又は損失(△),1000000,5843.0
6,previous,I 前第1四半期連結累計期間(自 2021年3月1日 至 2021年5月31日),2021-03-01,2021-05-31,3,4,百貨店・専門店事業,0,6,Sales,営業収益,1000000,166636.0
7,previous,I 前第1四半期連結累計期間(自 2021年3月1日 至 2021年5月31日),2021-03-01,2021-05-31,3,4,百貨店・専門店事業,1,7,Profit,セグメント利益又は損失(△),1000000,-3442.0
8,previous,I 前第1四半期連結累計期間(自 2021年3月1日 至 2021年5月31日),2021-03-01,2021-05-31,4,5,金融関連事業,0,6,Sales,営業収益,1000000,49101.0
9,previous,I 前第1四半期連結累計期間(自 2021年3月1日 至 2021年5月31日),2021-03-01,2021-05-31,4,5,金融関連事業,1,7,Profit,セグメント利益又は損失(△),1000000,10431.0


エラーログは何もありません。

In [21]:
errors

[]

以下の企業のHTMLをあらかじめダウンロード済みです。任意の企業の `path` をコピーして、 `exercise_path` として設定し実行してみてください。

In [22]:
html_paths = pd.DataFrame([str(p).replace("\\", "/") for p in Path("../data/raw/HTML").glob("*/qualitative.htm")], columns=["path"])
html_paths

,path
0,../data/raw/HTML/あおぞら銀行/qualitative.htm
1,../data/raw/HTML/アサヒグループホールディングス/qualitative.htm
2,../data/raw/HTML/アスクル/qualitative.htm
3,../data/raw/HTML/アドバンスト/qualitative.htm
4,../data/raw/HTML/イオン北海道/qualitative.htm
5,../data/raw/HTML/ウェザーニューズ/qualitative.htm
6,../data/raw/HTML/エーザイ/qualitative.htm
7,../data/raw/HTML/オリンパス/qualitative.htm
8,../data/raw/HTML/コジマ/qualitative.htm
9,../data/raw/HTML/サッポロホールディングス/qualitative.htm


In [23]:
exercise_path = "../data/raw/HTML/アサヒグループホールディングス/qualitative.htm"

正確に読み取りできれば `exercise_df` に読み取り結果が、読み取りできなければ `exception` にエラーの内容が格納されます。

In [24]:
exercise_df = None
exception = None

try:
    exercise_df, exercise_errors = financial_result_to_dataframe(exercise_path)
except FileNotFoundError as ex:
    exception = ex
except SegmentNotFoundException as ex:
    exception = ex
except ReadSegmentTableException as ex:
    exception = ex

In [25]:
exercise_df

In [26]:
exception

前期、当期2つのテーブルが取得できませんでした。
0番目のテーブル: 0番目のテーブルでは売上・利益の勘定が取得できませんでした
※N番目のテーブルのHTML要素は `exception.errors[N].table` でアクセスできます。

読み取れなかった HTML については、 `exception` にエラーの原因が出力されます。どのような HTML が読み取れて、どのような HTML が読み取れないのか、検証をしてみてください。この検証から、 HTML 化している発行体の方に、どのように開示頂ければ有効に開示情報が使われるのか、そのヒントが得られるはずです。

In [27]:
_error_html = ""
if isinstance(exception, ReadSegmentTableException):
    _error_html = str(exception.errors[0].table)

HTML(_error_html)

検証結果は `notebooks/HomeworkTemplate.ppt` の Power Point にまとめてください。

<img src="images/homework.png" width=600 alt="homework.png">

実際の決算短信を利用したい場合は、[TDnet適時開示情報閲覧サービス](https://www.release.tdnet.info/inbs/I_main_00.html)か、[東証上場会社情報サービス](https://www2.jpx.co.jp/tseHpFront/JJK010010Action.do?Show=Show)から開示情報のXBRLをダウンロードし、取り出したHTMLファイル(`qualitative.htm`)を`data/raw`フォルダの中に配置してください。そのあと、`exercise_path`を指定すればダウンロード済みのものと同じように読み込めます。

![how_to_download_001.png](images/how_to_download_001.png)

![how_to_download_002.png](images/how_to_download_002.png)
